This module implements HI-LO card-counting.

This works by assigning point values of (-1, 0, 1) to the denominations of cards ({2, 3, ,4, 5, 6}, {7 , 8, 9}, {10, J, Q, K, A}), the running count. 

In this module, the true count is evaluated to be (running count) / (no. unseen decks).

At the start of each round, the player will place her bet, according to the true count that has been updated from the end of the previous round.

Betting behaviour needs to be refined, but for now, we will use the slapstick 20x bet if TC > 2; and 1x bet if TC < 2.

Implementation consists of two modules.

* The player_bet module uses the true count to vary the player bets, as defined above.
* An important initial edge case that needs to be specified is when no cards have been dealt, when bets are placed on the 1st round of a particular shoe.


* The update_truecount module computes the true count at the end of a round, after all cards have been dealt. It takes the player and dealer's hand as input; and previous round running count and total cards dealt to compute the true count.

* The updated running count, total cards dealt and true count are then stored for use in the next round.


* A feature needs to be added at the control flow level whereby the true count, running count and total cards dealt are reset to 0 after the shoe has been shuffled (i.e. the replenish shoe module has been triggered).


* We need to emphasise that the new true count is computed and updated using the player hand, the dealer hand, the running count, and the total cards dealt.

Integration into CF:

* Need to amend the way that payouts and losses are computed - as we will no longer be using constant bets per round, but rather, a bet informed by the true count.

* Need to write this to account for multiway splits also, but shouldn't be too difficult.

In [16]:
import pandas as pd
import random
from matplotlib import pyplot 
import numpy as np

In [17]:
def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

In [18]:
def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

In [19]:
def openinghand():
    
    """Deals two cards to the player, and one to the dealer from the shuffled shoe. Outputs a message if there is Blackjack
    
    Cards are taken sequentially from the shuffled card shoe, beginning with the last element of the shuffled shoe list object
    and working in reverse order."""
    
    player = []
    dealer = []

    player.append(shuffled_shoe.pop())
    player.append(shuffled_shoe.pop())
    
    dealer.append(shuffled_shoe.pop())
    
    if ('A' in player) and (('10' in player) or ('J' in player) or ('Q' in player) or ('K' in player)):
        player_blackjack = 1
        print("Blackjack, house pays out at 3:2")
    else:
        player_blackjack = 0
        print("No Blackjack this time")
        
    return player, dealer, player_blackjack

In [20]:
deck, total_cards = getcards(decks=1)
shuffled_shoe = shuffler(deck, total_cards)
bet_per_round = 5

truecount = 0
total_cards_dealt = 0
runningcount = 0

In [316]:
current_hand_bet = player_bets(truecount, total_cards_dealt)
player, dealer, player_blackjack = openinghand()
truecount, runningcount, total_cards_dealt = update_truecount(player, dealer, runningcount, total_cards_dealt)

As this is the first round, there is no available statistical information to use to place bets.
I will therefore bet 5
Blackjack, house pays out at 3:2
The running count is -3, and the number of unseen decks is 0.9423076923076923, giving a true count of -3.183673469387755


In [317]:
print(player, dealer)

['10', 'A'] ['A']


In [34]:
def player_bets(truecount, total_cards_dealt):
    
    """A specification of player betting behaviour. Uses the truecount from the previous round to inform betting behaviour.
    If it is the first round of a shoe, then the minimum bet is placed."""
    
    if total_cards_dealt == 0:
        current_hand_bet = bet_per_round
        print("As this is the first round of the shoe, there is no available statistical information to use to place bets.")
        print("I will therefore bet {}".format(current_hand_bet))
    else:
        if (truecount <= 2):
            current_hand_bet = bet_per_round
            print("As the true count is {}, I will bet {}".format(truecount, current_hand_bet))
        if truecount > 2:
            current_hand_bet = 20 * bet_per_round
            print("As the true count is {}, I will bet {}".format(truecount, current_hand_bet))
    return current_hand_bet

In [32]:
def update_truecount(player, dealer, runningcount, total_cards_dealt):
    
    """Updates the true count, running count, and the number of cards dealt; at the end of the round. This information is used
    to inform the player's bet in the next round."""
    
    print("At the beginning of the round, the running count was {}, and the total number of cards dealt was {}".format(runningcount, total_cards_dealt))
    
    cards_dealt = player + dealer
    
    current_round_running_count = 0 
    
    for card in cards_dealt:
        if card in ['2','3','4','5','6']:
            current_round_running_count += 1
        if card in ['7','8','9']:
            current_round_running_count += 0
        if card in ['10','J', 'Q', 'K', 'A']:
            current_round_running_count -= 1
            
    runningcount = current_round_running_count + runningcount
            
    current_round_cards_dealt = len(cards_dealt)
    total_cards_dealt += current_round_cards_dealt
    total_unseen_cards = total_cards - total_cards_dealt
    unseen_decks = total_unseen_cards / 52 
    truecount = runningcount / unseen_decks
    
    print("The running count change for this round is {}".format(current_round_running_count))
    
    print("At the end of the round, the running count is now {}, and the number of unseen decks is {}, giving a true count of {}".format(runningcount, unseen_decks, truecount))
    
    return truecount, runningcount, total_cards_dealt

In [33]:
def multiple_hand_update_truecount(player_split_hands, dealer, runningcount, total_cards_dealt):
    
    """Updates the true count, running count and the number of cards dealt; at the end of round, for every hand."""
    
    print("At the beginning of the round, the running count was {}, and the total number of cards dealt was {}".format(runningcount, total_cards_dealt))
    
    cards_dealt = []
    
    for hand in player_split_hands:
        cards_dealt += hand
        
    cards_dealt += dealer
    
    current_round_running_count = 0
    
    for card in cards_dealt:
        if card in ['2','3','4','5','6']:
            current_round_running_count += 1
        if card in ['7','8','9']:
            current_round_running_count += 0
        if card in ['10','J', 'Q', 'K', 'A']:
            current_round_running_count -= 1
            
    runningcount = current_round_running_count + runningcount
            
    current_round_cards_dealt = len(cards_dealt)
    total_cards_dealt += current_round_cards_dealt
    total_unseen_cards = total_cards - total_cards_dealt
    unseen_decks = total_unseen_cards / 52 
    truecount = runningcount / unseen_decks
    
    print("The running count change for this round is {}".format(current_round_running_count))
    
    print("The running count is {}, and the number of unseen decks is {}, giving a true count of {}".format(runningcount, unseen_decks, truecount))
    
    return truecount, runningcount, total_cards_dealt

In [5]:
def clear_count_variables(truecount, runningcount, total_cards_dealt):
    
    truecount = 0
    runningcount = 0
    total_cards_dealt = 0
    
    return truecount, runningcount, total_cards_dealt

In [4]:
def replenish_shoe(threshold):
    
    """Checks whether or not  we are near the end of a shoe. If so, discards the remaining cards in the current shoe,
    collects them together with remaining cards that were discarded in previous hands, reshuffles all of them togther,
    and finally replenishes the shoe."""
    
    global deck
    global total_cards
    global shuffled_shoe
    global truecount
    global runningcount
    global total_cards_dealt
    
    fraction_cards_remaining = len(shuffled_shoe) / total_cards
    
    if fraction_cards_remaining < threshold:
        shuffled_shoe.clear
        deck, total_cards = getcards(decks=6)
        shuffled_shoe = shuffler(deck, total_cards)
        print("As we are reaching near the end of the shoe, we will reshuffle")
        truecount, runningcount, total_cards_dealt = clear_count_variables(truecount, runningcount, total_cards_dealt)
        print("As we have reshuffled, all card-counting variables are reset")
    else:
        print("No need for reshuffling yet, as card threshold not reached")

    return shuffled_shoe

In [7]:
def payout_loss(hand_outcome, decision):
    
    """Uses the outcome to calculate payoffs/losses, and decision of whether player has doubled down
    to update player's capital stock accordingly. """
    
    global capital_stock
    
    if hand_outcome == 0:
        if decision == 'D':
            new_capital = capital_stock[-1] - (2 * current_hand_bet)
            capital_stock.append(new_capital)
            print("Player loses " + str(2 * current_hand_bet) + " pounds from earlier doubling down.")
        else:
            new_capital = capital_stock[-1] - current_hand_bet
            capital_stock.append(new_capital)
            print("Player loses " + str(current_hand_bet) + " pounds.")
    
    elif hand_outcome == 1:
        if decision == 'D':
            new_capital = capital_stock[-1] + (2 * current_hand_bet)
            capital_stock.append(new_capital)
            print("Player wins " + str(2 * current_hand_bet) + " pounds from earlier doubling down.")
        else:
            new_capital = capital_stock[-1] + current_hand_bet
            capital_stock.append(new_capital)
            print("Player wins  " + str(current_hand_bet) + " pounds")
        
    elif hand_outcome == 2:
        new_capital = capital_stock[-1]
        capital_stock.append(new_capital)
        print("Draw, no change to player's initial capital.")
        
    else:
        new_capital = capital_stock[-1] + (1.5 * current_hand_bet)
        capital_stock.append(new_capital)
        print("Player blackjacks, house pays out " + str(1.5 * current_hand_bet) + " pounds.")
        
    print("Player now has" + " " + str(capital_stock[-1]) + " " + "pounds remaining.")
        
    return capital_stock

In [8]:
def multiple_payout_loss(multiple_hand_outcome_list, decision_list):
    
    global capital_stock
    
    for hand_outcome, decision in zip(multiple_hand_outcome_list, decision_list):
    
        capital_stock = payout_loss(hand_outcome, decision)
    
    return capital_stock

In [31]:
multiple_hand_update_truecount([['2','2'],['2','2'],['2','2']], ['2','2'], 0, 8)

The running count is 8, and the number of unseen decks is 0.6923076923076923, giving a true count of 11.555555555555555


(11.555555555555555, 8, 16)